# Download Sentinel-2 Data from EarthDaily Analytics' Earth Platform

This notebook provides a guide for downloading [Sentinel-2](https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2) data from the [EarthDaily Analytics Earth Platform](https://earthplatform.eds.earthdaily.com/) and displaying some of the data downloaded from this platform.

Ideally, this is a template for downloading Sentinel-2 data for an area **you** are interested in learning more about and inspiring **you** to perform your own analysis based on your own personal whimsy.

We start by introducing techniques for querying the Platform by using a GeoJSON Polygon created for the Lower Mainland; recommend how you can go about creating your own; and then proceed with an example for querying and displaying data from this store.

### What you Need to Run this Notebook:

(1) A file located in the same directory as this notebook called `.env` containing the following format and values:

```bash
CLIENT_ID="very_real_client_id_for_earthdaily_analytics_platform"
CLIENT_SECRET="very_real_client_secret_for_earthdaily_analytics_platform"
AUTH_TOKEN_URL="very_real_url_for_connecting_to_earthdaily_platform"
API_URL="very_real_api_url_for_reaching_earthdaily_dataset"
```

These values should be provided to you, but if you do not have them, please ask someone in the know.

(2) jazz hands?

Now that we have that squared away we are going to start by installing some required Python packages for this notebook.

In [ ]:
# pip install folium fsspec geopandas mapclassify python-dotenv pystac-client shapely

In [1]:
from datetime import datetime, timedelta
from pathlib import Path
import json
import os
import requests
import typing as T
import glob
import shutil
import csv
import sys

from dotenv import load_dotenv
from pystac_client import Client
from pystac.item import Item
from shapely.geometry import shape
from shapely.geometry import Polygon
from shapely.prepared import prep
from shapely.geometry import MultiPolygon, Point
import rasterio
from rasterio.features import rasterize
from rasterio.windows import Window
from dateutil.parser import parse

import cv2
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile as tiff
from PIL import Image
import ast
import imagecodecs
import torch


## Areas of Interest

To start, we need to create an area of interest that we want to analyze further.

If you have an area in mind already that you want to explore and learn more about, you can create your own GeoJSON file at [geojson.io](https://geojson.io/) or using an open source service like [QGIS](https://qgis.org/en/site/).

You can also download pre-built Shapefiles from services like [OpenStreetMap](https://download.geofabrik.de/) or from some of the resources mentioned in [this post by Carleton University](https://library.carleton.ca/find/gis/geospatial-data/shapefiles-canada-united-states-and-world).

We are going to start with a GeoJSON polygon that I've pre-built using geojson.io's interface that covers the some of Vancouver, BC, Canada and will inspect some Sentinel-2 tiles from the EarthDaily platform.

In [2]:


# Example GeoJSON geometry
geojson_geometry = {
    # I drew a Polygon at geojson.io and put the contents of the `geometry` attribute here
    "type": "Polygon",
    "coordinates": [
        [
            [-123.19443904574592, 49.24932487550103],
            [-123.17802762978633, 49.13635374192958],
            [-123.03251301430745, 49.04966338761224],
            [-122.73054282797881, 49.06113525720673],
            [-122.68349674929038, 49.13420622071877],
            [-122.77868300152072, 49.255752174748864],
            [-122.98984330819309, 49.29073056863652],
            [-123.19443904574592, 49.24932487550103],
        ]
    ],
}

# Convert the GeoJSON geometry into a shapely geometry
# shapely_geometry = shape(geojson_geometry)

# Please Note: You can also create, download, or import your own Shapefile to use with GeoPandas
# and create a GeoDataFrame from it. To do this see the docs at:
# https://geopandas.org/en/stable/docs/user_guide/io.html#reading-spatial-data
# and take a look at the geopandas read_file method.

# Create a GeoDataFrame with the shapely geometry
# Note that the Coordinate Reference System (CRS) defines how the two-dimensional,
# projected geometry relates to real world locations.
# gdf = gpd.GeoDataFrame([{"geometry": shapely_geometry}], crs="EPSG:4326")
gdf = gpd.read_file('../shapefiles/campbell_river.shp')
gdf = gdf.set_crs("EPSG:4326")
shapely_geometry = gdf.geometry[0]
gdf = gdf[gdf.geometry.is_valid]
aoi = gdf[gdf.geometry.is_valid].unary_union

# Calculate the centroid of your GeoDataFrame to center the map
centroid = gdf.geometry.centroid.unary_union.centroid

# Create a folium map centered on the centroid of your shapefile
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=10)

# Add the GeoDataFrame as a layer to the folium map
folium.GeoJson(gdf, name="geojson").add_to(m)

# Add layer control to toggle the geojson layer
folium.LayerControl().add_to(m)

# Display the map
m

/var/folders/8t/jw61zz954rbcbps32kpn49l80000gn/T/ipykernel_3062/955933354.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid.unary_union.centroid


## Downloading Data

To download Sentinel-2 data from the Earth Platform we are going to use the [SpatioTemporal Asset Catalog (STAC)](https://stacspec.org/en) interface - a common language used to describe geospatial information. To do this, we need to authenticate with this platform using our precious credentials and then we can use the GeoDataFrame object we've already created to download data from the platform.

We demonstrate how to authorization token from this service in the following cell.

In [15]:
# load_dotenv(
#     # You will need to create this file with your
#     # CLIENT_ID, CLIENT_SECRET, AUTH_TOKEN_URL, and API_URL
#     dotenv_path=".env"
# )

CLIENT_ID = os.environ.get("CLIENT_ID")
CLIENT_SECRET = os.environ.get("CLIENT_SECRET")
AUTH_TOKEN_URL = os.environ.get("AUTH_TOKEN_URL")
API_URL = os.environ.get("API_URL")


def get_new_token(client_id: str, client_secret: str, auth_token_url: str):
    """
    Authenticate with the Earth Platform and obtain a new access token.
    """
    token_req_payload = {"grant_type": "client_credentials"}
    token_response = requests.post(
        auth_token_url,
        data=token_req_payload,
        allow_redirects=False,
        auth=(client_id, client_secret),
    )
    token_response.raise_for_status()  # Raise an exception if the request failed

    tokens = json.loads(token_response.text)
    return tokens["access_token"]


token = get_new_token(CLIENT_ID, CLIENT_SECRET, AUTH_TOKEN_URL)
# Open a client to the STAC API
client = Client.open(API_URL, headers={"Authorization": f"Bearer {token}"})

## Getting overlapping tiles

Currently, we compare the satellite tiles with the selected area's polygon to identify the intersecting regions. These intersecting areas are then divided into smaller, regularly sized square tiles.

In [4]:
def load_canada_map() -> gpd.GeoDataFrame:
    # CRS is EPSG:4326
    gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    return gdf[gdf.name == "Canada"]

def grid_bounds(geom, delta):
    # Convert a larger shapefile into grids...
    # Logic retrieved from:
    # https://www.matecdev.com/posts/shapely-polygon-gridding.html
    minx, miny, maxx, maxy = geom.bounds
    nx = int((maxx - minx)/delta)
    ny = int((maxy - miny)/delta)
    gx, gy = np.linspace(minx,maxx,nx), np.linspace(miny,maxy,ny)
    grid = []
    for i in range(len(gx)-1):
        for j in range(len(gy)-1):
            poly_ij = Polygon([[gx[i],gy[j]],[gx[i],gy[j+1]],[gx[i+1],gy[j+1]],[gx[i+1],gy[j]]])
            grid.append(poly_ij)

    return grid

def partition(geom, delta):
    prepared_geom = prep(geom)
    grid = list(filter(prepared_geom.intersects, grid_bounds(geom, delta)))
    return grid

In [5]:
areas_geojson = load_canada_map()

polygons = partition(areas_geojson.iloc[0].geometry, 5)

/var/folders/8t/jw61zz954rbcbps32kpn49l80000gn/T/ipykernel_3062/4144051380.py:3: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [6]:
def get_intersecting_polygons(polygons, aoi):
    """
    Given an area with larger multipolygons, and a group of polygons return a list of
    polygons that intersect with the area of interest.
    """
    intersecting_polygons = []
    for polygon in polygons:
        if aoi.intersects(polygon):
            intersecting_polygons.append(polygon)
    return intersecting_polygons

wanted_polygons = get_intersecting_polygons(polygons, aoi)
len(polygons), len(wanted_polygons)

(91, 2)

## Query the STAC API

Now that we have established a connection to the client, we can use the geometry we've selected and obtain Sentinel-2 tiles for that area of interest. We are going to start by returning the available items for our area of interest within our query constraints and will then decide which tiles we want to download for further use.

In [7]:
def remove_small_tiles(
    gdf: gpd.GeoDataFrame, min_area: float = 1e6, reproject: bool = True
):
    """
    Given a GeoDataFrame as input, remove all geometries that are less than the specified area.

    Returns:
        gdf: A GeoDataFrame with geometries removed.
    """
    gdf_projected = gdf.copy()
    if reproject:
        lcc_crs = "+proj=lcc +lat_1=40 +lat_2=65 +lon_0=125 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
        gdf_projected = gdf.to_crs(lcc_crs)

    gdf_projected["area_km"] = gdf_projected["geometry"].area / 1000
    gdf["proj_area"] = gdf_projected["area_km"]
    gdf = gdf.loc[gdf["proj_area"] > min_area]

    return gdf

def add_geometries_iteratively(
    gdf: gpd.GeoDataFrame, intersection_threshold: float = 0.95, debug: bool = False
) -> tuple[MultiPolygon, gpd.GeoDataFrame]:
    """
    Given a GeoDataFrame as input, construct an area iteratively by adding all of the geometries together.

    Returns:
        merged_geometry: A shapely Polygon or MultiPolygon object representing the merged geometry.
        selected_geometries: A list of GeoDataFrame rows that were selected to be merged.
    """
    assert intersection_threshold < 1, "The intersection threshold must be less than 1."
    # Initialize an empty geometry
    merged_geometry = None
    selected_geometries = []

    # Iterate over each row in the GeoDataFrame
    for idx, row in gdf.iterrows():
        geometry = row["geometry"]
        intersected = False  # Flag to track if the row has intersected geometry

        # Check if the current geometry is a MultiPolygon
        if isinstance(geometry, MultiPolygon):
            # Iterate over each polygon in the MultiPolygon
            for polygon in geometry.geoms:
                if merged_geometry is None:
                    merged_geometry = polygon
                else:
                    if (
                        merged_geometry.intersection(polygon).area / polygon.area
                    ) > intersection_threshold:
                        intersected = True
                    else:
                        merged_geometry = merged_geometry.union(polygon)
        else:
            # If the current geometry is not a MultiPolygon
            if merged_geometry is None:
                merged_geometry = geometry
            else:
                if (
                    merged_geometry.intersection(geometry).area / geometry.area
                ) > intersection_threshold:
                    intersected = True
                else:
                    merged_geometry = merged_geometry.union(geometry)

        # Print a message if the row has no non-intersecting area
        if intersected:
            if debug:
                print(
                    f"Row {idx} has no area that does not already intersect with the merged geometry."
                )
        else:
            selected_geometries.append(row)

    return (merged_geometry, gpd.GeoDataFrame(selected_geometries, crs=gdf.crs))

def get_all_overlapping_tiles(polygons, start_date, end_date):
    print(f"Getting area for: {len(polygons)} polygons")
    all_items, all_gdfs = [], []
    for polygon in polygons:
        poly_obj = {
            "type": "Polygon",
            "coordinates": list(polygon.__geo_interface__["coordinates"])
        }
        items, tile_gdf = get_sentinel2_data(client, poly_obj, start_date, end_date)
        if len(items) == 0:
            print("No items found for given area... Not great.")
            continue


        tile_gdf = remove_small_tiles(tile_gdf, reproject=True)
        _, tile_gdf = add_geometries_iteratively(tile_gdf)

        wanted_gdf = tile_gdf[tile_gdf.intersects(aoi)]

        wanted_tiles = [name.split("/")[-1] for name in wanted_gdf["earthsearch:s3_path"].tolist()]
        wanted_items = [item for item in items if item.id in wanted_tiles]
        all_items.append(wanted_items)
        all_gdfs.append(wanted_gdf)


    return all_items, all_gdfs

def get_sentinel2_data(
    client: Client,
    aoi: dict,
    start_date: str,
    end_date: str,
    cloud_cover: float = 10,
    max_items: int = 500,
):
    """
    Download Sentinel-2 data from the Earth Platform.
    """
    query = client.search(
        collections=["sentinel-2-l2a"],
        datetime=f"{start_date}T00:00:00.000000Z/{end_date}T00:00:00.000000Z",  # 2023-07-10T00:00:00.000000Z/2023-07-20T00:00:00.000000Z
        intersects=aoi,  # The area of interest; you can also query by bbox, or other geometry
        query={"eo:cloud_cover": {"lte": cloud_cover}},
        sortby=[
            {
                "field": "properties.eo:cloud_cover",
                "direction": "asc",
            },  # Sort by cloud cover from lowest to highest
        ],
        limit=max_items,  # This is the number of items to be returned per page
        max_items=max_items,  # This is number of items to page over
    )

    items = list(query.items())
    if len(items) == 0:
        raise Exception(
            "No items found, try enlarging search area or increasing cloud cover threshold."
        )
    print(f"Found: {len(items):d} tiles.")

    # Convert STAC items into a GeoJSON FeatureCollection
    stac_json = query.item_collection_as_dict()
    gdf = gpd.GeoDataFrame.from_features(stac_json, crs="EPSG:4326")

    return items, gdf

In [ ]:
sentinel_items, sentinel_gdf = get_all_overlapping_tiles(wanted_polygons, start_date, end_date)
print(f"Found {len(sentinel_items)} Sentinel-2 items/tiles.")
# combine all gdf in sentinel_gdf to a single gdf
sentinel_gdf = gpd.GeoDataFrame(pd.concat(sentinel_gdf), crs=sentinel_gdf[0].crs)
# combine all items in sentinel_items to a single list
sentinel_items = [item for sublist in sentinel_items for item in sublist]

In [ ]:
# Take a look at the tiles found for the given query on the map
sentinel_gdf.explore(color="green")

In [ ]:
# The items object contains a list of STAC items which includes
# metadata about the satellite imagery and links to the actual data.
# You can access the first item in the list like this:
sentinel_items[0]

## Download our tiles

Now we want to download our Sentinel-2 tiles from EarthDaily's Earth Store. In this example we are going to download all of the high, medium, and low resolution bands that are captured by the Sentinel-2 satellites. To learn a bit more about the significance of each captured band we are downloading, see this tutorial [here](https://gisgeography.com/sentinel-2-bands-combinations/).

Note that the information we download in the below cells does not exhaust the available data for each tile. You can find links to different files like class labels and metadata on the tile by downloading more files found for a given Sentinel-2 item.

In [8]:
# This cell creates utility functions to download the files associated with a STAC item to a local
# file system.

def download_file(href: str, outpath: Path):
    """
    Given a URL, download the file to the specified path.
    """
    with requests.get(href, stream=True) as r:
        r.raise_for_status()
        with open(outpath, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)


def download_files_for_item(
    item: Item, asset_dict: dict[str, str], outpath: Path, debug: bool = True
) -> bool:
    """
    Save all files of interest for a given item.

    If one file fails to download return False, otherwise return True.
    """
    if not outpath.exists():
        outpath.mkdir(exist_ok=True, parents=True)

    for key, value in asset_dict.items():
        if debug:
          print(f"Downloading {key} and relabeling to {value}")
        if key in item.assets:
            if key in ["tileinfo_metadata"]:
                file_outpath = outpath / f"{value}.json"
            else:
                file_outpath = outpath / f"{value}.tiff"
            if not file_outpath.exists():
                try:
                    download_file(item.assets[key].href, file_outpath)
                except requests.ConnectionError:
                    print(
                        f"Failed to download {item.assets[key].href} for item {item.id}"
                    )
                    return False
                except requests.exceptions.ReadTimeout:
                    print(
                        f"Experienced a read timeout for {item.assets[key].href} for item {item.id}"
                    )
                    return False
            else:
                print(f"Skipping {item.assets[key].href} as it already exists.")

    return True

In [9]:
# Spatial Resolution Data Found Here: https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/resolutions/spatial
# 10 meter resolution bands, order is important for Max resolution bands, B04=Red, B03=Green, B02=Blue, B08=NIR
MAX_RESOLUTION = ["B04", "B03", "B02", "B08"]
MID_RESOLUTION = [
    "B05",
    "B06",
    "B07",
    "B8A",
    "B11",
    "B12",
    # "scl",
]  # 20 meter resolution bands
MIN_RESOLUTION = ["B01", "B09"]  # 60 meter resolution bands
# MIN_RESOLUTION = ["B01", "B09", "aot"]  # 60 meter resolution bands

MIN_RESOLUTION_SIZE = 128
MID_RESOLUTION_SIZE = 384
MAX_RESOLUTION_SIZE = 768

EXTRACTED_BANDS = [
    ("B02", 10),
    ("B03", 10),
    ("B04", 10),
    ("B08", 10),
    ("B05", 20),
    ("B06", 20),
    ("B07", 20),
    ("B8A", 20),
    ("B11", 20),
    ("B12", 20),
    # ("scl", 20),
    ("B01", 60),
    ("B09", 60),
    # ("aot", 60),
]

# This ordering is based on the order of the bands read into the dataloader
# Basically - read 10m, 20m, 60m bands in that order
BAND_ORDERING = [
    "B04",
    "B03",
    "B02",
    "B08",
    "B05",
    "B06",
    "B07",
    "B8A",
    "B11",
    "B12",
    "B01",
    "B09",
]

def get_band_specification(filepath: Path) -> tuple[str, int]:
    """
    Get the metadata from a Sentinel-2 file.
    """
    for band in MIN_RESOLUTION:
        if band == filepath.stem:
            return (band, MIN_RESOLUTION_SIZE)
    for band in MID_RESOLUTION:
        if band == filepath.stem:
            return (band, MID_RESOLUTION_SIZE)
    for band in MAX_RESOLUTION:
        if band == filepath.stem:
            return (band, MAX_RESOLUTION_SIZE)

    return ("", 0)

In [10]:
def get_aoi_shape(gdf: gpd.GeoDataFrame, target_crs: str) -> MultiPolygon:
    return gdf.to_crs(target_crs).unary_union


def create_masks(gdf: Path, out_path: Path, bounding_box: Polygon, meta: dict):
    gdf = gdf[gdf.intersects(bounding_box)]
    if len(gdf) == 0:
        raise ValueError("Intersecting area must have at least one polygon.")
    geometries = [(geom, value) for geom, value in zip(gdf.geometry, gdf.label)]

    with rasterio.open(out_path, "w+", **meta) as dest:
        out_arr = dest.read(1)
        burned = rasterize(
            geometries,
            out=out_arr,
            transform=dest.transform,
            fill=0,
            default_value=0,
            dtype=rasterio.uint8,
        )
        dest.write_band(1, burned)

def create_coordinates_file(bound: list, original_crs: gpd.GeoDataFrame.crs, segment_output_file: Path):
    '''
    Create a json file with the coordinates of the segment

    Args:
    bound: list of coordinates of the segment
    segment_output_file: Path to the segment file

    Returns:
    None
    '''
    tile_geometry = [Point(coord) for coord in bound]

    tile_gdf = gpd.GeoDataFrame(geometry=tile_geometry)
    tile_gdf.crs = "EPSG:32609"

    tile_gdf = tile_gdf.to_crs(original_crs)

    # get a list of the coordinates of the segment
    coordinates = list(list(point.coords)[0] for point in tile_gdf.geometry)

    geojson_dict = {
    "geometry": {
        "coordinates": [coordinates]
    }
    }
    coordinate_file = segment_output_file.parent / "coordinates.json"
    if not os.path.exists(coordinate_file):
        # file does not exist, create json file
        with open(coordinate_file, 'w') as f:
            json.dump(geojson_dict, f)

def generate_tiles(
    input_file: Path,
    output_dir: Path,
    band_name: str,
    window_size: int = 768,
    aoi_gdf: T.Optional[gpd.GeoDataFrame] = None,
) -> int:
    """
    Split Sentinel files into square tiles of a given window size.
    """
    all_bounds = []
    with rasterio.open(input_file.as_posix()) as src:
        height, width = src.shape

        # Calculate the number of segments in both dimensions
        num_rows = (height + window_size - 1) // window_size
        num_cols = (width + window_size - 1) // window_size
        num_processed = 0
        skipped = 0
        blank = 0
        misshapen = 0
        no_mask = 0
        aoi_gdf["label"] = 1
        original_crs = aoi_gdf.crs
        aoi_gdf = aoi_gdf.to_crs(src.crs)
        aoi = aoi_gdf.unary_union

        for row in range(num_rows):
            for col in range(num_cols):
                row_start = row * window_size
                col_start = col * window_size

                # Calculate the actual segment size based on the remaining pixels
                seg_height = min(window_size, height - row_start)
                seg_width = min(window_size, width - col_start)
                seg_window = Window(col_start, row_start, seg_width, seg_height)
                # skip if window is not a perfect square or the shape of the window is not the same as the window size
                if seg_height != seg_width or seg_height != window_size:
                    skipped += 1
                    misshapen += 1
                    continue

                seg_data = src.read(window=seg_window)
                seg_profile = src.profile.copy()
                seg_transform = src.window_transform(
                    seg_window
                )  # don't use rasterio.windows.transform...

                # Create a Polygon for the window's bounds
                bound = [
                    seg_transform * (0, 0),
                    seg_transform * (seg_window.width, 0),
                    seg_transform * (seg_window.width, seg_window.height),
                    seg_transform * (0, seg_window.height),
                ]
                polygon = Polygon(bound)

                if not polygon.intersects(aoi):
                    no_mask += 1
                    continue

                all_bounds.append(polygon)
                seg_profile.update(
                    width=seg_height,
                    height=seg_width,
                    transform=seg_transform,
                    compress="lzw",
                )

                segment_output_file = output_dir / f"{row}_{col}" / f"{band_name}.tif"
                segment_output_file.parent.mkdir(parents=True, exist_ok=True)


                create_coordinates_file(bound, original_crs, segment_output_file)

                with rasterio.open(
                    segment_output_file, "w", **seg_profile
                ) as segment_dst:
                    segment_dst.write(seg_data)

                mask_path = output_dir / f"{row}_{col}" / "mask.tif"
                if not mask_path.exists() and window_size == 768:
                    create_masks(aoi_gdf, mask_path, polygon, seg_profile)

                num_processed += 1

        print(
            f"Processed tiles: {num_processed}, blank tiles: {blank}, misshapen tiles: {misshapen}, no_mask: {no_mask}"
        )

        return num_processed, skipped, all_bounds

In [11]:
high_resolution_bands = {"red": "B04", "green": "B03", "blue": "B02", "nir": "B08"}
mid_resolution_bands = {
    "rededge1": "B05",
    "rededge2": "B06",
    "rededge3": "B07",
    "nir08": "B8A",
    "swir16": "B11",
    "swir22": "B12",
}
low_resolution_bands = {"coastal": "B01", "nir09": "B09"}
other_files = {
    "scl": "scl",  # Scene Classification Map
    "tileinfo_metadata": "metadata",  # Tile Metadata
}
all_download_files = { # Modify this variable to change the files that are downloaded
    **high_resolution_bands,
    **mid_resolution_bands,
    **low_resolution_bands,
    **other_files,
}


In [12]:
def download_and_tile_files(
    gdf: gpd.GeoDataFrame,
    items: list[Item],
    download_files: dict[str, str],
    aoi_gdf: gpd.GeoDataFrame,
    output_dir: Path):
    """
    Given a GeoDataFrame of items and a list of STAC items, download the
    files to a given output directory.

    Parameters:
      items: A list of items that correspond to tiles found in the GeoDataFrame and include paths to files to be
        downloaded in this function.
      download_files: A dictionary of strings where the keys correspond to names of items on the Earth Platform
        and the values correspond to their Sentinel-2 name.
    """
    gdf["downloaded"] = False
    downloaded = 0
    for index, tile in enumerate(items):
        dt_obj = datetime.strptime(tile.properties["datetime"], "%Y-%m-%dT%H:%M:%S.%fZ")
        formatted_date = dt_obj.strftime("%Y%m%d")
        out_path = output_dir / tile.id / formatted_date
        downloaded = download_files_for_item(tile, download_files, out_path)

        if downloaded:
            gdf.loc[
                gdf["s2:granule_id"] == tile.properties["s2:granule_id"], "downloaded"
            ] = True
            for file in out_path.iterdir():
                band_name, window_size = get_band_specification(file)
                if band_name and window_size:
                    out_dir = file.parent / "tiles"
                    if not out_dir.exists():
                        out_dir.mkdir(parents=True)
                    generate_tiles(file, out_dir, band_name, window_size, aoi_gdf)

        if downloaded:
            downloaded += 1
        else:
            print(f"Unable to download file for item with id: {tile.id} at index: {index} in items list.")

    print(
        f"Downloaded all bands for {downloaded} tiles. Failed to download at least one "
        + f"band or file for {len(items) - downloaded} tiles."
    )


# output_dir = Path("../content/sentinel_tiles") # Used for Google CoLab

# # We are only going to download 2 tiles here, but feel free to modify this function
# # call to download more data!
# download_and_tile_files(sentinel_gdf[0:2], sentinel_items[0:2], all_download_files, gdf, output_dir)

In [16]:
date_list = [
#  ['2017-04-17', '2017-04-26'],
#  ['2017-05-09', '2017-05-22'],
#  ['2017-05-25', '2017-06-08'],
#  ['2017-06-13', '2017-06-27'],
#  ['2017-06-29', '2017-07-12'],
#  ['2017-07-15', '2017-07-30'],
#  ['2017-07-31', '2017-08-14'],
#  ['2017-08-17', '2017-09-01'],
#  ['2017-09-02', '2017-09-17'],
#  ['2017-09-18', '2017-10-03'],
#  ['2017-10-05', '2017-10-20'],
#  ['2017-10-23', '2017-11-02'],
#  ['2018-03-12', '2018-03-20'],
#  ['2018-04-15', '2018-04-26'],
#  ['2018-05-05', '2018-05-20'],
#  ['2018-05-22', '2018-06-06'],
#  ['2018-06-09', '2018-06-23'],
#  ['2018-06-25', '2018-07-07'],
#  ['2018-07-11', '2018-07-26'],
#  ['2018-07-27', '2018-08-11'],
#  ['2018-08-12', '2018-08-25'],
#  ['2018-08-28', '2018-09-06'],
# ##  ['2018-09-13', '2018-09-13'],
#  ['2018-09-29', '2018-10-14'],
#  ['2018-10-19', '2018-10-27'],
#  ['2019-03-18', '2019-04-01'],
#  ['2019-04-24', '2019-05-09'],
#  ['2019-05-10', '2019-05-24'],
#  ['2019-05-27', '2019-06-11'],
#  ['2019-06-13', '2019-06-28'],
#  ['2019-06-29', '2019-07-14'],
#  ['2019-07-15', '2019-07-29'],
#  ['2019-08-03', '2019-08-18'],
#  ['2019-08-19', '2019-09-03'],
#  ['2019-09-04', '2019-09-18'],
#  ['2019-09-20', '2019-10-05'],
#  ['2019-10-08', '2019-10-14'],
## ['2019-10-30', '2019-10-30'],
## ['2019-11-28', '2019-11-28'],
#  ['2020-03-16', '2020-03-29'],
#  ['2020-04-09', '2020-04-22'],
#  ['2020-04-25', '2020-05-08'],
# ## ['2020-06-07', '2020-06-16'],
#  ['2020-06-30', '2020-07-12'],
#  ['2020-07-24', '2020-07-28'],
#  ['2020-07-29', '2020-07-31'],
#  ['2020-08-11', '2020-08-17'],
#  ['2020-08-27', '2020-09-10'],
#  ['2020-09-18', '2020-09-22'],
## ['2020-10-30', '2020-10-30'],
## ['2020-12-21', '2020-12-21'],
#  ['2021-04-01', '2021-04-16'],
#  ['2021-04-17', '2021-04-25'],
#  ['2021-05-05', '2021-05-20'],
#  ['2021-05-21', '2021-06-05'],
#  ['2021-06-08', '2021-06-22'],
#  ['2021-06-25', '2021-07-09'],
#  ['2021-07-11', '2021-07-26'],
#  ['2021-07-28', '2021-08-12'],
#  ['2021-08-15', '2021-08-27'],
 ['2021-09-02', '2021-09-14'],
## ['2021-09-22', '2021-09-22'],
## ['2021-11-03', '2021-11-05'],
## ['2021-12-11', '2021-12-13'],
## ['2023-01-24', '2023-01-24']
 ]

downloaded_dates = [['2022-02-06', '2022-02-17'],
 ['2022-03-09', '2022-03-09'],
 ['2022-04-07', '2022-04-07'],
 ['2022-05-26', '2022-05-29'],
 ['2022-06-28', '2022-07-11'],
 ['2022-07-14', '2022-07-26'],
 ['2022-07-31', '2022-08-15'],
 ['2022-08-17', '2022-09-01'],
 ['2022-09-02', '2022-09-17'],
 ['2022-09-19', '2022-10-04'],
 ['2022-10-05', '2022-10-20'],
 ['2022-10-22', '2022-11-05'],
 ['2022-11-08', '2022-11-20'],
 ['2022-11-29', '2022-11-29']]

In [17]:
for start_date, end_date in date_list:
    print("--------------------")
    print(f"Start date: {start_date}")
    print(f"End date: {end_date}")
    print("--------------------")
    print(f"Dropping {len(gdf) - len(gdf[gdf.geometry.is_valid])} invalid geometries.")
    gdf = gdf[gdf.geometry.is_valid]
    aoi = gdf[gdf.geometry.is_valid].unary_union

    all_items, all_gdfs = get_all_overlapping_tiles(wanted_polygons, start_date, end_date)
    if len(all_items) == 0:
        continue

    # all_gdfs = gpd.GeoDataFrame(pd.concat(all_gdfs), crs=all_gdfs[0].crs)
    items = [item for sublist in all_items for item in sublist] # covers western canada
    gdfs = pd.concat(all_gdfs)
    print(f"Number of items: {len(items)}")
    print(f"Number of gdfs: {len(gdfs)}")

    output_dir = Path("../datasets/sentinel2")
    download_and_tile_files(gdfs, items, all_download_files, gdf, output_dir)

--------------------
Start date: 2021-07-11
End date: 2021-07-26
--------------------
Dropping 0 invalid geometries.
Getting area for: 2 polygons
Found: 47 tiles.
Found: 197 tiles.
Number of items: 18
Number of gdfs: 18
Processed tiles: 55, blank tiles: 0, misshapen tiles: 29, no_mask: 141
Processed tiles: 55, blank tiles: 0, misshapen tiles: 29, no_mask: 141
Processed tiles: 55, blank tiles: 0, misshapen tiles: 29, no_mask: 141
Processed tiles: 55, blank tiles: 0, misshapen tiles: 29, no_mask: 141
Processed tiles: 55, blank tiles: 0, misshapen tiles: 29, no_mask: 141
Processed tiles: 55, blank tiles: 0, misshapen tiles: 29, no_mask: 141
Processed tiles: 55, blank tiles: 0, misshapen tiles: 29, no_mask: 141
Processed tiles: 55, blank tiles: 0, misshapen tiles: 29, no_mask: 141
Processed tiles: 55, blank tiles: 0, misshapen tiles: 29, no_mask: 141
Processed tiles: 55, blank tiles: 0, misshapen tiles: 29, no_mask: 141
Processed tiles: 55, blank tiles: 0, misshapen tiles: 29, no_mask: 141

## Organize wildfire data
Organize wildfire data by time and location

In [126]:
fire_df.drop(fire_df.index, inplace=True)

In [154]:
# Get all filenames in the directory into a list
filenames = os.listdir('../datasets/bc_fire_points/')

# Read all the csv files into one dataframe
fire_df_1 = pd.concat([pd.read_csv('../datasets/bc_fire_points/' + f) for f in filenames[1:]])

fire_df_2 = pd.read_csv('../datasets/bc_fire_points/' + filenames[0])


In [155]:
# drop columns that are not needed
fire_df_1 = fire_df_1.drop(["Fire Number", "X", "Y"], axis=1)

# Merge the two columns into one
fire_df_1['coordinates'] = fire_df_1['LONGITUDE'].astype(str) + ', ' + fire_df_1['LATITUDE'].astype(str)

# Change name Ignition Data to date
fire_df_1 = fire_df_1.rename(columns={"IGNITION DATE": "date"})

fire_df_1['date'] = pd.to_datetime(fire_df_1['date'], errors='coerce')
fire_df_1 = fire_df_1.dropna()

# Convert the date column to YYYYMMDD
fire_df_1['date'] = fire_df_1['date'].dt.strftime('%Y%m%d')

# drop columns that are not needed
fire_df_1 = fire_df_1.drop(["LATITUDE", "LONGITUDE"], axis=1)
fire_df_1

/var/folders/8t/jw61zz954rbcbps32kpn49l80000gn/T/ipykernel_55789/753379810.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fire_df_1['date'] = pd.to_datetime(fire_df_1['date'], errors='coerce')


,date,coordinates
0,20210421,"-123.824717, 51.48205"
1,20210626,"-121.962133, 52.050917"
2,20210709,"-120.414017, 53.4259"
3,20210828,"-122.7818, 52.520117"
4,20210802,"-121.535912, 52.139225"
...,...,...
1866,20191030,"-126.0737, 49.286"
1867,20190626,"-125.7094, 49.3239"
1868,20190626,"-125.4045, 49.346"
1869,20190626,"-125.4014, 49.3429"


In [156]:
# drop columns that are not needed
fire_df_2 = fire_df_2.drop(["Fire Number", "X", "Y"], axis=1)

# Merge the two columns into one
fire_df_2['coordinates'] = fire_df_2['LONGITUDE'].astype(str) + ', ' + fire_df_2['LATITUDE'].astype(str)

# Change name Ignition Data to date
fire_df_2 = fire_df_2.rename(columns={"IGNITION DATE": "date"})

fire_df_2['date'] = pd.to_datetime(fire_df_2['date'], errors='coerce')
fire_df_2 = fire_df_2.dropna()

# Convert the date column to YYYYMMDD
fire_df_2['date'] = fire_df_2['date'].dt.strftime('%Y%m%d')

# drop columns that are not needed
fire_df_2 = fire_df_2.drop(["LATITUDE", "LONGITUDE"], axis=1)
fire_df_2

,date,coordinates
0,20220717,"-121.019467, 52.623"
1,20220824,"-122.375417, 51.0147"
2,20220823,"-118.872217, 49.543983"
3,20220812,"-120.567867, 50.472167"
4,20220823,"-121.4897, 49.221417"
...,...,...
1796,20220822,"-126.947533, 54.082667"
1797,20220714,"-121.446867, 49.782967"
1798,20220819,"-126.409033, 54.18675"
1799,20230116,"-118.743783, 52.43955"


In [158]:
# Combine the two dataframes
fire_df = pd.concat([fire_df_2, fire_df_1])
fire_df

,date,coordinates
0,20220717,"-121.019467, 52.623"
1,20220824,"-122.375417, 51.0147"
2,20220823,"-118.872217, 49.543983"
3,20220812,"-120.567867, 50.472167"
4,20220823,"-121.4897, 49.221417"
...,...,...
1866,20191030,"-126.0737, 49.286"
1867,20190626,"-125.7094, 49.3239"
1868,20190626,"-125.4045, 49.346"
1869,20190626,"-125.4014, 49.3429"


In [159]:
fire_df.to_csv('../dataset_tables/fire_points.csv', index=False)

In [19]:
fire_df = pd.read_csv('../dataset_tables/fire_points.csv')

In [160]:
# Create a polygon by a list of coordinates
coordinates_list = [[-128.54948032,51.37397123],
    [-123.42622011591264, 51.26222303279471],
    [-123.52937300599798, 48.20558993286168],
    [-128.3913125586851, 48.7163322815431]]
wanted_polygon = Polygon(coordinates_list)

# Create a column called points
fire_df['points'] = fire_df['coordinates'].apply(lambda x: Point(float(x.split(',')[0]), float(x.split(',')[1])))

# filter column points by the polygon
fire_df = fire_df[fire_df['points'].apply(lambda x: wanted_polygon.contains(x))]

# sort the dataframe by date
fire_df = fire_df.sort_values(by='date')

In [20]:
fire_df

,date,coordinates
0,20220717,"-121.019467, 52.623"
1,20220824,"-122.375417, 51.0147"
2,20220823,"-118.872217, 49.543983"
3,20220812,"-120.567867, 50.472167"
4,20220823,"-121.4897, 49.221417"
...,...,...
9089,20191030,"-126.0737, 49.286"
9090,20190626,"-125.7094, 49.3239"
9091,20190626,"-125.4045, 49.346"
9092,20190626,"-125.4014, 49.3429"


## Organize satellite file names into dataframe

This script will move downloaded images to `prepared_dataset`. The process is briefed as follows:

- Images downloaded will be stored in `download_file` folder.
- Under `download_file` folder, images will be grouped according to the polygon they belong to.
- In each polygon, 2 types of images are present: geotiff with band information, and a mask file.
- Geotiffs should be moved and stored in `prepared_dataset/images_directory{group_id}` folder.
- Mask files are stored in `prepare_dataset/mask_directory{group_id}` folder.

In [162]:
raw_df.drop(fire_df.index, inplace=True)

In [21]:
def fast_scandir(dirname: str) -> list:
    """
    Scan and return all subfolders of a directory.
    """
    subfolders= [f.path for f in os.scandir(dirname) if f.is_dir()]
    for dirname in list(subfolders):
        subfolders.extend(fast_scandir(dirname))
    return subfolders

def get_subfolders_with_keyword(keyword: str, subfolders_list: list) -> list:
    subfolders_with_keyword_list = []

    for folder in subfolders_list:
        if keyword in folder:
            subfolders_with_keyword_list.append(folder)

    return subfolders_with_keyword_list

In [23]:
source_path = "../datasets/sentinel2"
subfolders_list = fast_scandir(source_path)
print(f"Number of subfolders: {len(subfolders_list)}")

subfolders_with_keyword_list = get_subfolders_with_keyword("tiles/", subfolders_list) # note that we need the / to get folders
print(f"Number of subfolders with keyword: {len(subfolders_with_keyword_list)}")

Number of subfolders: 4212
Number of subfolders with keyword: 3885


In [24]:
# Organized the tiles information into a dataframe
raw_df = pd.DataFrame(subfolders_with_keyword_list, columns=["path_name"])
detailed_df = pd.DataFrame([x.rsplit('/') for x in raw_df['path_name']])

# insert detailed_df into raw_df
raw_df = pd.concat([raw_df, detailed_df], axis=1)
raw_df.drop([0, 1, 5], axis=1, inplace=True)

# rename columns
raw_df = raw_df.rename(columns={"path_name": "path_name_sentinel2", 2: "satellite", 3: "imagery_id", 4: "date", 6: "tile_id"})

# generate saving path name by data and tile_id starting with "..prepared_dataset"
raw_df['saving_path'] = raw_df.apply(lambda x: f"../prepared_dataset/{x['date']}/{x['tile_id']}", axis=1)

In [25]:
# open a file called coordinates.json from path_name_sentinel2
# save the content of the file into a new column called 'coordinates'

def get_coordinates_from_json(path_name_sentinel2: str) -> str:
    with open(f"{path_name_sentinel2}/coordinates.json", "r") as file:
        tile_info = json.load(file)
        coordinates = tile_info['geometry']['coordinates'][0]
        coordinates_str = ",".join(str(tuple(coord)) for coord in coordinates)
        return coordinates_str

raw_df['coordinates'] = raw_df.apply(lambda x: get_coordinates_from_json(x['path_name_sentinel2']), axis=1)

In [26]:
raw_df

,path_name_sentinel2,satellite,imagery_id,date,tile_id,saving_path,coordinates
0,../datasets/sentinel2/S2A_10UDU_20210627_0_L2A...,sentinel2,S2A_10UDU_20210627_0_L2A,20210627,3_3,../prepared_dataset/20210627/3_3,"(-130.0431945012966, 48.54101501413099),(-129...."
1,../datasets/sentinel2/S2A_10UDU_20210627_0_L2A...,sentinel2,S2A_10UDU_20210627_0_L2A,20210627,1_0,../prepared_dataset/20210627/1_0,"(-130.35896158173583, 48.675907349440955),(-13..."
2,../datasets/sentinel2/S2A_10UDU_20210627_0_L2A...,sentinel2,S2A_10UDU_20210627_0_L2A,20210627,0_4,../prepared_dataset/20210627/0_4,"(-129.94301766388796, 48.74915454749164),(-129..."
3,../datasets/sentinel2/S2A_10UDU_20210627_0_L2A...,sentinel2,S2A_10UDU_20210627_0_L2A,20210627,0_3,../prepared_dataset/20210627/0_3,"(-130.04747777245979, 48.7482523225942),(-129...."
4,../datasets/sentinel2/S2A_10UDU_20210627_0_L2A...,sentinel2,S2A_10UDU_20210627_0_L2A,20210627,2_0,../prepared_dataset/20210627/2_0,"(-130.35710701535717, 48.60683612543683),(-130..."
...,...,...,...,...,...,...,...
3880,../datasets/sentinel2/S2A_10UCB_20210418_0_L2A...,sentinel2,S2A_10UCB_20210418_0_L2A,20210418,8_10,../prepared_dataset/20210418/8_10,"(-130.75149652634843, 50.88555899383295),(-130..."
3881,../datasets/sentinel2/S2A_10UCB_20210418_0_L2A...,sentinel2,S2A_10UCB_20210418_0_L2A,20210418,11_11,../prepared_dataset/20210418/11_11,"(-130.6351205097082, 50.68003573020781),(-130...."
3882,../datasets/sentinel2/S2A_9UWR_20210603_0_L2A/...,sentinel2,S2A_9UWR_20210603_0_L2A,20210603,0_12,../prepared_dataset/20210603/0_12,"(-127.69955084215027, 50.54503064537831),(-127..."
3883,../datasets/sentinel2/S2A_9UWR_20210603_0_L2A/...,sentinel2,S2A_9UWR_20210603_0_L2A,20210603,0_13,../prepared_dataset/20210603/0_13,"(-127.59119010461468, 50.543769930790944),(-12..."


In [120]:
raw_df.iloc[0]['coordinates']

'(-125.7607154787941, 50.29994118390697),(-125.65306262838969, 50.29688850061719),(-125.65789652279332, 50.22793508254463),(-125.7653945658079, 50.23098034961218)'

## Merge folder names for the satellite and fire information

In [27]:
# convert the string coordinates into list of coordinates
raw_df['coordinates'] = raw_df['coordinates'].apply(lambda x: [list(coord) for coord in ast.literal_eval(x)])
fire_df['coordinates'] = fire_df['coordinates'].apply(lambda x: [float(coord) for coord in ast.literal_eval(x)])

# Convert coordinates list in raw_df to Polygon
raw_df['coordinates'] = raw_df['coordinates'].apply(lambda x: Polygon(x))

# Convert coordinates list in fire_df to Point
fire_df['coordinates'] = fire_df['coordinates'].apply(lambda x: Point(x))

# Convert date(int) to datetime(YYYY-MM-DD)
fire_df['date'] = pd.to_datetime(fire_df['date'], format='%Y%m%d')
raw_df['date'] = pd.to_datetime(raw_df['date'], format='%Y%m%d')

In [28]:
fire_df

,date,coordinates
0,2022-07-17,POINT (-121.019467 52.623)
1,2022-08-24,POINT (-122.375417 51.0147)
2,2022-08-23,POINT (-118.872217 49.543983)
3,2022-08-12,POINT (-120.567867 50.472167)
4,2022-08-23,POINT (-121.4897 49.221417)
...,...,...
9089,2019-10-30,POINT (-126.0737 49.286)
9090,2019-06-26,POINT (-125.7094 49.3239)
9091,2019-06-26,POINT (-125.4045 49.346)
9092,2019-06-26,POINT (-125.4014 49.3429)


In [29]:
# Create a new column in raw_df called "if_fire" and set it to None
raw_df['if_fire'] = False
raw_df.head()

,path_name_sentinel2,satellite,imagery_id,date,tile_id,saving_path,coordinates,if_fire
0,../datasets/sentinel2/S2A_10UDU_20210627_0_L2A...,sentinel2,S2A_10UDU_20210627_0_L2A,2021-06-27,3_3,../prepared_dataset/20210627/3_3,POLYGON ((-130.0431945012966 48.54101501413099...,False
1,../datasets/sentinel2/S2A_10UDU_20210627_0_L2A...,sentinel2,S2A_10UDU_20210627_0_L2A,2021-06-27,1_0,../prepared_dataset/20210627/1_0,POLYGON ((-130.35896158173583 48.6759073494409...,False
2,../datasets/sentinel2/S2A_10UDU_20210627_0_L2A...,sentinel2,S2A_10UDU_20210627_0_L2A,2021-06-27,0_4,../prepared_dataset/20210627/0_4,POLYGON ((-129.94301766388796 48.7491545474916...,False
3,../datasets/sentinel2/S2A_10UDU_20210627_0_L2A...,sentinel2,S2A_10UDU_20210627_0_L2A,2021-06-27,0_3,../prepared_dataset/20210627/0_3,POLYGON ((-130.04747777245979 48.7482523225942...,False
4,../datasets/sentinel2/S2A_10UDU_20210627_0_L2A...,sentinel2,S2A_10UDU_20210627_0_L2A,2021-06-27,2_0,../prepared_dataset/20210627/2_0,POLYGON ((-130.35710701535717 48.6068361254368...,False


In [30]:
# compare the columns of data and coordinates(list) in fire_df
# with the columns of data and coordinates(Polygon) raw_df
# if the coordinates of fire_df is within the coordinates of raw_df
# then set the if_fire column to True
for index, row in fire_df.iterrows():
    for index2, row2 in raw_df.iterrows():
        point = row['coordinates']
        polygon = row2['coordinates']
        date = row['date']
        date2 = row2['date']
        if polygon.contains(point) and timedelta(days=0) <= date - date2 <= timedelta(days=2):
            raw_df.at[index2, 'if_fire'] = True
            print(f"fire time: {date} and tile time: {date2}.")

fire time: 2021-06-28 00:00:00 and tile time: 2021-06-27 00:00:00.
fire time: 2021-05-13 00:00:00 and tile time: 2021-05-13 00:00:00.
fire time: 2021-06-30 00:00:00 and tile time: 2021-06-28 00:00:00.
fire time: 2021-06-30 00:00:00 and tile time: 2021-06-28 00:00:00.
fire time: 2021-06-30 00:00:00 and tile time: 2021-06-28 00:00:00.
fire time: 2021-07-12 00:00:00 and tile time: 2021-07-12 00:00:00.
fire time: 2021-06-30 00:00:00 and tile time: 2021-06-28 00:00:00.
fire time: 2021-04-19 00:00:00 and tile time: 2021-04-18 00:00:00.
fire time: 2021-06-30 00:00:00 and tile time: 2021-06-28 00:00:00.


In [31]:
raw_df['if_fire'].value_counts()

if_fire
False    3876
True        9
Name: count, dtype: int64

In [32]:
raw_df.to_csv('../dataset_tables/raw_df2.csv', index=False)

In [33]:
# Randomly select 5% of the rows in each month where 'if_fire' is False
# and merge these rows with the rows where 'if_fire' is True

raw_df['month'] = raw_df['date'].dt.month

# For rows where 'if_fire' is False, group by month and randomly select 5% of the rows in each group
df_false = raw_df[raw_df['if_fire'] == False].groupby('month').apply(lambda x: x.sample(frac=0.01))

# Get the rows where 'if_fire' is True
df_true = raw_df[raw_df['if_fire'] == True]

# Merge the two dataframes
df_sentinel2 = pd.concat([df_false, df_true])

df_sentinel2.drop(columns=['month'], inplace=True)

df_sentinel2

/var/folders/8t/jw61zz954rbcbps32kpn49l80000gn/T/ipykernel_3062/4279838260.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_false = raw_df[raw_df['if_fire'] == False].groupby('month').apply(lambda x: x.sample(frac=0.01))


,path_name_sentinel2,satellite,imagery_id,date,tile_id,saving_path,coordinates,if_fire
"(4, 2056)",../datasets/sentinel2/S2B_10UCB_20210413_1_L2A...,sentinel2,S2B_10UCB_20210413_1_L2A,2021-04-13,12_11,../prepared_dataset/20210413/12_11,POLYGON ((-130.63272561095894 50.6109972212777...,False
"(4, 1785)",../datasets/sentinel2/S2A_10UDU_20210418_1_L2A...,sentinel2,S2A_10UDU_20210418_1_L2A,2021-04-18,2_0,../prepared_dataset/20210418/2_0,POLYGON ((-130.35710701535717 48.6068361254368...,False
"(4, 302)",../datasets/sentinel2/S2B_9UXR_20210413_0_L2A/...,sentinel2,S2B_9UXR_20210413_0_L2A,2021-04-13,1_10,../prepared_dataset/20210413/1_10,POLYGON ((-126.5092231510031 50.45656499061053...,False
"(4, 1225)",../datasets/sentinel2/S2A_10UCU_20210418_1_L2A...,sentinel2,S2A_10UCU_20210418_1_L2A,2021-04-18,2_13,../prepared_dataset/20210418/2_13,POLYGON ((-130.3587344156348 48.60681693325166...,False
"(4, 2413)",../datasets/sentinel2/S2A_9UXR_20210418_1_L2A/...,sentinel2,S2A_9UXR_20210418_1_L2A,2021-04-18,4_10,../prepared_dataset/20210418/4_10,POLYGON ((-126.52003704052083 50.2495497923371...,False
"(4, 2664)",../datasets/sentinel2/S2A_9UYQ_20210418_1_L2A/...,sentinel2,S2A_9UYQ_20210418_1_L2A,2021-04-18,1_8,../prepared_dataset/20210418/1_8,POLYGON ((-125.38750127772715 49.5272233112519...,False
"(4, 2960)",../datasets/sentinel2/S2B_10UCA_20210413_0_L2A...,sentinel2,S2B_10UCA_20210413_0_L2A,2021-04-13,10_0,../prepared_dataset/20210413/10_0,POLYGON ((-131.7809675056921 49.82820360027362...,False
"(4, 1977)",../datasets/sentinel2/S2B_10UCV_20210413_0_L2A...,sentinel2,S2B_10UCV_20210413_0_L2A,2021-04-13,0_0,../prepared_dataset/20210413/0_0,POLYGON ((-131.76906793699982 49.6195968745351...,False
"(4, 2406)",../datasets/sentinel2/S2A_9UXR_20210418_1_L2A/...,sentinel2,S2A_9UXR_20210418_1_L2A,2021-04-18,7_12,../prepared_dataset/20210418/7_12,POLYGON ((-126.31643949143137 50.0377655632022...,False
"(4, 1697)",../datasets/sentinel2/S2B_9UYR_20210413_0_L2A/...,sentinel2,S2B_9UYR_20210413_0_L2A,2021-04-13,8_5,../prepared_dataset/20210413/8_5,POLYGON ((-125.67704558452314 49.9521098104475...,False


In [34]:
df_sentinel2['if_fire'].value_counts()

if_fire
False    39
True      9
Name: count, dtype: int64

In [35]:
def split_data(df):
    train_size = int(len(df) * 0.6)
    test_size = int(len(df) * 0.2)

    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    df['split'] = 'val'

    df.loc[:train_size, 'split'] = 'train'
    df.loc[train_size:train_size+test_size, 'split'] = 'test'

    return df

df_true = df_sentinel2[df_sentinel2['if_fire'] == True]
df_false = df_sentinel2[df_sentinel2['if_fire'] == False]

df_sentinel2 = pd.concat([split_data(df_true), split_data(df_false)])

In [36]:
df_sentinel2

,path_name_sentinel2,satellite,imagery_id,date,tile_id,saving_path,coordinates,if_fire,split
0,../datasets/sentinel2/S2A_9UYR_20210627_0_L2A/...,sentinel2,S2A_9UYR_20210627_0_L2A,2021-06-27,12_10,../prepared_dataset/20210627/12_10,POLYGON ((-125.16477804389937 49.6598726202365...,True,train
1,../datasets/sentinel2/S2B_9UWR_20210628_1_L2A/...,sentinel2,S2B_9UWR_20210628_1_L2A,2021-06-28,0_13,../prepared_dataset/20210628/0_13,POLYGON ((-127.59119010461468 50.5437699307909...,True,train
2,../datasets/sentinel2/S2B_9UXS_20210628_1_L2A/...,sentinel2,S2B_9UXS_20210628_1_L2A,2021-06-28,11_2,../prepared_dataset/20210628/11_2,POLYGON ((-127.3671432187832 50.68006747113423...,True,train
3,../datasets/sentinel2/S2B_9UXR_20210513_1_L2A/...,sentinel2,S2B_9UXR_20210513_1_L2A,2021-05-13,5_9,../prepared_dataset/20210513/5_9,POLYGON ((-126.63108549894987 50.1827852845182...,True,train
4,../datasets/sentinel2/S2A_9UYR_20210418_1_L2A/...,sentinel2,S2A_9UYR_20210418_1_L2A,2021-04-18,12_1,../prepared_dataset/20210418/12_1,POLYGON ((-126.12112636240417 49.6876281074014...,True,train
5,../datasets/sentinel2/S2B_9UYR_20210712_0_L2A/...,sentinel2,S2B_9UYR_20210712_0_L2A,2021-07-12,8_8,../prepared_dataset/20210712/8_8,POLYGON ((-125.35650387832956 49.9424725439986...,True,test
6,../datasets/sentinel2/S2B_9UXS_20210628_1_L2A/...,sentinel2,S2B_9UXS_20210628_1_L2A,2021-06-28,12_3,../prepared_dataset/20210628/12_3,POLYGON ((-127.26103949458563 50.6094595867684...,True,test
7,../datasets/sentinel2/S2B_9UWS_20210628_1_L2A/...,sentinel2,S2B_9UWS_20210628_1_L2A,2021-06-28,13_13,../prepared_dataset/20210628/13_13,POLYGON ((-127.59115795420932 50.5448487890229...,True,val
8,../datasets/sentinel2/S2B_9UXS_20210628_1_L2A/...,sentinel2,S2B_9UXS_20210628_1_L2A,2021-06-28,13_0,../prepared_dataset/20210628/13_0,POLYGON ((-127.5886182856442 50.54481803086994...,True,val
0,../datasets/sentinel2/S2B_10UDU_20210818_1_L2A...,sentinel2,S2B_10UDU_20210818_1_L2A,2021-08-18,2_1,../prepared_dataset/20210818/2_1,POLYGON ((-130.25294992462867 48.6080166014167...,False,train


In [37]:
# move column split to the front
cols = list(df_sentinel2.columns)
cols = [cols[-1]] + cols[:-1]
df_sentinel2 = df_sentinel2[cols]

In [38]:
df_sentinel2

,split,path_name_sentinel2,satellite,imagery_id,date,tile_id,saving_path,coordinates,if_fire
0,train,../datasets/sentinel2/S2A_9UYR_20210627_0_L2A/...,sentinel2,S2A_9UYR_20210627_0_L2A,2021-06-27,12_10,../prepared_dataset/20210627/12_10,POLYGON ((-125.16477804389937 49.6598726202365...,True
1,train,../datasets/sentinel2/S2B_9UWR_20210628_1_L2A/...,sentinel2,S2B_9UWR_20210628_1_L2A,2021-06-28,0_13,../prepared_dataset/20210628/0_13,POLYGON ((-127.59119010461468 50.5437699307909...,True
2,train,../datasets/sentinel2/S2B_9UXS_20210628_1_L2A/...,sentinel2,S2B_9UXS_20210628_1_L2A,2021-06-28,11_2,../prepared_dataset/20210628/11_2,POLYGON ((-127.3671432187832 50.68006747113423...,True
3,train,../datasets/sentinel2/S2B_9UXR_20210513_1_L2A/...,sentinel2,S2B_9UXR_20210513_1_L2A,2021-05-13,5_9,../prepared_dataset/20210513/5_9,POLYGON ((-126.63108549894987 50.1827852845182...,True
4,train,../datasets/sentinel2/S2A_9UYR_20210418_1_L2A/...,sentinel2,S2A_9UYR_20210418_1_L2A,2021-04-18,12_1,../prepared_dataset/20210418/12_1,POLYGON ((-126.12112636240417 49.6876281074014...,True
5,test,../datasets/sentinel2/S2B_9UYR_20210712_0_L2A/...,sentinel2,S2B_9UYR_20210712_0_L2A,2021-07-12,8_8,../prepared_dataset/20210712/8_8,POLYGON ((-125.35650387832956 49.9424725439986...,True
6,test,../datasets/sentinel2/S2B_9UXS_20210628_1_L2A/...,sentinel2,S2B_9UXS_20210628_1_L2A,2021-06-28,12_3,../prepared_dataset/20210628/12_3,POLYGON ((-127.26103949458563 50.6094595867684...,True
7,val,../datasets/sentinel2/S2B_9UWS_20210628_1_L2A/...,sentinel2,S2B_9UWS_20210628_1_L2A,2021-06-28,13_13,../prepared_dataset/20210628/13_13,POLYGON ((-127.59115795420932 50.5448487890229...,True
8,val,../datasets/sentinel2/S2B_9UXS_20210628_1_L2A/...,sentinel2,S2B_9UXS_20210628_1_L2A,2021-06-28,13_0,../prepared_dataset/20210628/13_0,POLYGON ((-127.5886182856442 50.54481803086994...,True
0,train,../datasets/sentinel2/S2B_10UDU_20210818_1_L2A...,sentinel2,S2B_10UDU_20210818_1_L2A,2021-08-18,2_1,../prepared_dataset/20210818/2_1,POLYGON ((-130.25294992462867 48.6080166014167...,False


## Organize satellite files by tiles

In [39]:
def fast_scandir(dirname: str) -> list:
    """
    Scan and return all subfolders of a directory.

    Parameters:
    dirname: root directory name to be scanned.

    Returns:
    A list of subfolders under the given directory.
    """
    subfolders = [f.path for f in os.scandir(dirname) if f.is_dir()]
    for dirname in list(subfolders):
        subfolders.extend(fast_scandir(dirname))
    return subfolders


def get_folders_with_keyword(keyword: str, folders_list: list) -> list:
    """
    From a list of folders, cherry pick the ones with given keyword.

    Parameters:
    keyword: keyword that folders directory must contain.
    folders_list: list of folders to be searched.

    Returns:
    A list of paths to folders that contain the given keyword.
    """
    folders_with_keyword_list = []

    for folder in folders_list:
        if keyword in folder:
            folders_with_keyword_list.append(folder)

    return folders_with_keyword_list


def get_list_of_files_in_directory(directory_name: str, keyword: str = ".tif") -> list:
    """
    Given a folder directory, get a list of files with certain keywords.
    For the usecase, we search for ".tif" files.

    Parameters:
    directory_name: name of the folder directory.
    keyword: keyword the files inside the directory must contain.

    Returns:
    A list of files whose names contain the specified keyword, under the specified directory.
    """
    return [f"{directory_name}/{f}" for f in os.listdir(directory_name) if f.endswith(keyword)]


def move_file(source_path: str, label: str, id: int) -> None:
    """
    Move one file from source to destination. The destination folder is separated
    based on label, date, and id. The moved file will retain its original name.

    Parameters:
    source_path: path to the file to be moved.
    label: either "image" or "mask".
    id: suffix to the destination folder name.
    """
    file_name = source_path.split("/")[-1]
    file_date = source_path.split("/")[-4]
    destination_folder = f"../prepared_dataset/{label}_directory_{file_date}_{id}"
    destination_path = os.path.join(destination_folder, file_name)


    if not os.path.isdir(destination_folder):
        os.makedirs(destination_folder)

    if os.path.isfile(destination_path):
        print("File exists.")
        return

    shutil.copy(source_path, destination_path)
    print(f"File copied to destination: {destination_path}.")

def get_path_str(source_path: str, label: str, id: int) -> str:
    """
    Get the new file name after moving the file.

    Parameters:
    source_path: path to the file to be moved.
    label: either "image" or "mask".
    id: suffix to the destination folder name.
    """
    file_name = source_path.split("/")[-1]
    file_date = source_path.split("/")[-4]
    path_name = f"{label}_directory_{file_date}_{id}"

    return path_name


def batch_move_files(source_path_list: list, df: pd) -> None:
    """
    Move files from a list of source paths. In this use case, images under `tiles` folder from `download_file`
    are moved to `prepare_dataset` folder. Images that are originally inside the same directory are grouped
    using the same id. Band geotif's are put under `image_directory{id}` while mask tif's are put under
    `mask_directory{id}`.

    Parameters:
    source_path_list: list of source paths.
    """
    path_dict = {}

    for i in range(len(source_path_list)):
        current_path = source_path_list[i]
        current_folder = current_path.rsplit(
            "/", 1)[0]  # split on the last occurrence

        if current_folder not in path_dict:
            path_dict[current_folder] = len(path_dict)
            df.loc[df['path_name_sentinel2'] == current_folder, 'data_path'] = get_path_str(current_path, "image", path_dict[current_folder])
            df.loc[df['path_name_sentinel2'] == current_folder, 'mask_path'] = get_path_str(current_path, "mask", path_dict[current_folder])

        current_id = path_dict[current_folder]

        if "mask" in current_path:
            move_file(current_path, "mask", current_id)
        else:
            move_file(current_path, "image", current_id)

    return df


In [40]:
# Get all folders from download_file folder
# source_path = os.path.join(os.path.dirname(__file__), "../datasets/sentinel2")
# subfolders_list = fast_scandir(source_path)

# Get subfolders_list from the path_name_sentinel2 column
subfolders_list = df_sentinel2['path_name_sentinel2'].tolist()

# Get folders that are under tiles folder
# note that we need the / to get folders
subfolders_with_keyword_list = get_folders_with_keyword(
    "tiles/", subfolders_list)

all_files = []
for subfolder in subfolders_with_keyword_list:
    current_list = get_list_of_files_in_directory(subfolder)
    all_files.extend(current_list)

# add two column to the dataframe
df_sentinel2['data_path'] = ''
df_sentinel2['mask_path'] = ''
df_sentinel2 = batch_move_files(all_files, df_sentinel2)

/var/folders/8t/jw61zz954rbcbps32kpn49l80000gn/T/ipykernel_3062/1075665750.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentinel2['data_path'] = ''
/var/folders/8t/jw61zz954rbcbps32kpn49l80000gn/T/ipykernel_3062/1075665750.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentinel2['mask_path'] = ''


File copied to destination: ../prepared_dataset/image_directory_20210627_0/B08.tif.
File copied to destination: ../prepared_dataset/image_directory_20210627_0/B09.tif.
File copied to destination: ../prepared_dataset/image_directory_20210627_0/B8A.tif.
File copied to destination: ../prepared_dataset/mask_directory_20210627_0/mask.tif.
File copied to destination: ../prepared_dataset/image_directory_20210627_0/B02.tif.
File copied to destination: ../prepared_dataset/image_directory_20210627_0/B03.tif.
File copied to destination: ../prepared_dataset/image_directory_20210627_0/B01.tif.
File copied to destination: ../prepared_dataset/image_directory_20210627_0/B04.tif.
File copied to destination: ../prepared_dataset/image_directory_20210627_0/B11.tif.
File copied to destination: ../prepared_dataset/image_directory_20210627_0/B05.tif.
File copied to destination: ../prepared_dataset/image_directory_20210627_0/B07.tif.
File copied to destination: ../prepared_dataset/image_directory_20210627_0/B

In [41]:
df_sentinel2

,split,path_name_sentinel2,satellite,imagery_id,date,tile_id,saving_path,coordinates,if_fire,data_path,mask_path
0,train,../datasets/sentinel2/S2A_9UYR_20210627_0_L2A/...,sentinel2,S2A_9UYR_20210627_0_L2A,2021-06-27,12_10,../prepared_dataset/20210627/12_10,POLYGON ((-125.16477804389937 49.6598726202365...,True,image_directory_20210627_0,mask_directory_20210627_0
1,train,../datasets/sentinel2/S2B_9UWR_20210628_1_L2A/...,sentinel2,S2B_9UWR_20210628_1_L2A,2021-06-28,0_13,../prepared_dataset/20210628/0_13,POLYGON ((-127.59119010461468 50.5437699307909...,True,image_directory_20210628_1,mask_directory_20210628_1
2,train,../datasets/sentinel2/S2B_9UXS_20210628_1_L2A/...,sentinel2,S2B_9UXS_20210628_1_L2A,2021-06-28,11_2,../prepared_dataset/20210628/11_2,POLYGON ((-127.3671432187832 50.68006747113423...,True,image_directory_20210628_2,mask_directory_20210628_2
3,train,../datasets/sentinel2/S2B_9UXR_20210513_1_L2A/...,sentinel2,S2B_9UXR_20210513_1_L2A,2021-05-13,5_9,../prepared_dataset/20210513/5_9,POLYGON ((-126.63108549894987 50.1827852845182...,True,image_directory_20210513_3,mask_directory_20210513_3
4,train,../datasets/sentinel2/S2A_9UYR_20210418_1_L2A/...,sentinel2,S2A_9UYR_20210418_1_L2A,2021-04-18,12_1,../prepared_dataset/20210418/12_1,POLYGON ((-126.12112636240417 49.6876281074014...,True,image_directory_20210418_4,mask_directory_20210418_4
5,test,../datasets/sentinel2/S2B_9UYR_20210712_0_L2A/...,sentinel2,S2B_9UYR_20210712_0_L2A,2021-07-12,8_8,../prepared_dataset/20210712/8_8,POLYGON ((-125.35650387832956 49.9424725439986...,True,image_directory_20210712_5,mask_directory_20210712_5
6,test,../datasets/sentinel2/S2B_9UXS_20210628_1_L2A/...,sentinel2,S2B_9UXS_20210628_1_L2A,2021-06-28,12_3,../prepared_dataset/20210628/12_3,POLYGON ((-127.26103949458563 50.6094595867684...,True,image_directory_20210628_6,mask_directory_20210628_6
7,val,../datasets/sentinel2/S2B_9UWS_20210628_1_L2A/...,sentinel2,S2B_9UWS_20210628_1_L2A,2021-06-28,13_13,../prepared_dataset/20210628/13_13,POLYGON ((-127.59115795420932 50.5448487890229...,True,image_directory_20210628_7,mask_directory_20210628_7
8,val,../datasets/sentinel2/S2B_9UXS_20210628_1_L2A/...,sentinel2,S2B_9UXS_20210628_1_L2A,2021-06-28,13_0,../prepared_dataset/20210628/13_0,POLYGON ((-127.5886182856442 50.54481803086994...,True,image_directory_20210628_8,mask_directory_20210628_8
0,train,../datasets/sentinel2/S2B_10UDU_20210818_1_L2A...,sentinel2,S2B_10UDU_20210818_1_L2A,2021-08-18,2_1,../prepared_dataset/20210818/2_1,POLYGON ((-130.25294992462867 48.6080166014167...,False,image_directory_20210818_9,mask_directory_20210818_9


In [42]:
# drop some columns of the dataframe
df_sentinel2 = df_sentinel2.drop(columns=["path_name_sentinel2", "satellite", "tile_id", "saving_path"])

In [43]:
df_sentinel2

,split,imagery_id,date,coordinates,if_fire,data_path,mask_path
0,train,S2A_9UYR_20210627_0_L2A,2021-06-27,POLYGON ((-125.16477804389937 49.6598726202365...,True,image_directory_20210627_0,mask_directory_20210627_0
1,train,S2B_9UWR_20210628_1_L2A,2021-06-28,POLYGON ((-127.59119010461468 50.5437699307909...,True,image_directory_20210628_1,mask_directory_20210628_1
2,train,S2B_9UXS_20210628_1_L2A,2021-06-28,POLYGON ((-127.3671432187832 50.68006747113423...,True,image_directory_20210628_2,mask_directory_20210628_2
3,train,S2B_9UXR_20210513_1_L2A,2021-05-13,POLYGON ((-126.63108549894987 50.1827852845182...,True,image_directory_20210513_3,mask_directory_20210513_3
4,train,S2A_9UYR_20210418_1_L2A,2021-04-18,POLYGON ((-126.12112636240417 49.6876281074014...,True,image_directory_20210418_4,mask_directory_20210418_4
5,test,S2B_9UYR_20210712_0_L2A,2021-07-12,POLYGON ((-125.35650387832956 49.9424725439986...,True,image_directory_20210712_5,mask_directory_20210712_5
6,test,S2B_9UXS_20210628_1_L2A,2021-06-28,POLYGON ((-127.26103949458563 50.6094595867684...,True,image_directory_20210628_6,mask_directory_20210628_6
7,val,S2B_9UWS_20210628_1_L2A,2021-06-28,POLYGON ((-127.59115795420932 50.5448487890229...,True,image_directory_20210628_7,mask_directory_20210628_7
8,val,S2B_9UXS_20210628_1_L2A,2021-06-28,POLYGON ((-127.5886182856442 50.54481803086994...,True,image_directory_20210628_8,mask_directory_20210628_8
0,train,S2B_10UDU_20210818_1_L2A,2021-08-18,POLYGON ((-130.25294992462867 48.6080166014167...,False,image_directory_20210818_9,mask_directory_20210818_9


In [44]:
df_sentinel2['if_fire'].value_counts()

if_fire
False    39
True      9
Name: count, dtype: int64

## Data process

After marking the data with fire occurrance, we can process the date to calculate the features we need.

In [45]:
def save_all_numerical_values_to_df(split, dataset):
    new_df = pd.DataFrame(columns=['ID', 'split', 'Label', 'NDVI', 'NBR', 'NDWI', 'NDBI', 'RGB'])
    # Loop through all the indices in the dataset
    for index in range(len(dataset)):
        numerical_values = dataset.get_numerical_values(index)
        if numerical_values:  # Check if the dictionary is not empty
            ndvi_list = numerical_values["NDVI"].flatten().tolist()
            nbr_list = numerical_values["NBR"].flatten().tolist()
            ndwi_list = numerical_values["NDWI"].flatten().tolist()
            ndbi_list = numerical_values["NDBI"].flatten().tolist()
            rgb_list = numerical_values["RGB"].flatten().tolist()
            location = numerical_values["Location"].replace(
                "image_directory_", "")

            # Add the values to the dataframe
            new_row = {
                "ID": location,
                "split": split,
                "Label": "",
                "NDVI": ndvi_list,
                "NBR": nbr_list,
                "NDWI": ndwi_list,
                "NDBI": ndbi_list,
                "RGB": rgb_list,

            }
            new_df = new_df._append(new_row, ignore_index=True)
    return new_df

In [46]:
import dataset

df = df_sentinel2[df_sentinel2['split'] == 'val']
sat_dataset = dataset.SATDataset(split='val',
                                data_path=Path("../prepared_dataset"),
                                df=df)
len(sat_dataset)

10

In [47]:
from importlib import reload

In [231]:
reload(dataset)

<module 'dataset' from '/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py'>

In [48]:
import dataset

# Check if foulder processed_images_folder exists
# if not create the folder
if not os.path.exists("../processed_images_folder"):
    os.makedirs("../processed_images_folder")

for split in ['test', 'train', 'val']:
    df = df_sentinel2[df_sentinel2['split'] == split]
    sat_dataset = dataset.SATDataset(split=split,
                                    data_path=Path("../prepared_dataset"),
                                    df=df)
    for index in range(len(sat_dataset)):
        # Get the filepaths for the current index
        filepath = sat_dataset.filepaths[index]

        # Extract the original directory name
        original_dir_name = Path(filepath).name

        # Get the transformed images
        images_dict = sat_dataset.get_images(
            index)  # Ensure you call the correct method

        # Create a new directory path for the processed images
        processed_dir_path_str = f"../processed_images_folder/{original_dir_name}"
        processed_dir_path = Path(processed_dir_path_str)
        processed_dir_path.mkdir(parents=True, exist_ok=True)

        for img_type, image in images_dict.items():
            # Skip saving the mask if you only want the indices and RGB images
            if img_type == "Mask":
                continue

            if img_type == "NBR":
                # Apply color mapping for NBR
                plt.imshow(image, cmap=plt.cm.RdYlGn, vmin=-1, vmax=1)
                plt.colorbar(orientation='vertical')
                plt.axis('off')

                # Save the current figure to a numpy array
                fig = plt.gcf()
                plt.draw()
                image_np = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
                image_np = image_np.reshape(
                    fig.canvas.get_width_height()[::-1] + (3,))

                # Convert numpy array to PIL Image
                pil_image = Image.fromarray(image_np)

                # Define the filename, replacing .tif with .png for NBR
                filename = f"{img_type}.png"

                # Close the figure to free memory
                plt.close(fig)
            else:
                # Normalize the image data to 0-255 for other image types
                # Clip to the range you want
                image = np.clip(image, 0, np.max(image))
                image_8bit = ((image - np.min(image)) /
                                (np.max(image) - np.min(image)) * 255).astype('uint8')

                # If the image has more than one channel, convert it to RGB
                if image_8bit.ndim > 2 and image_8bit.shape[2] > 3:
                    # Convert multi-band images (e.g., 4 bands) to RGB (3 bands) before saving as JPEG
                    image_8bit = image_8bit[:, :, :3]

                # Create the PIL Image from the numpy array
                pil_image = Image.fromarray(image_8bit)

                # Define the filename, replacing .tif with .jpg for other image types
                filename = f"{img_type}.jpg"

            # Define the full path for the file
            filepath = processed_dir_path / filename

            # Save the image
            pil_image.save(filepath)

/var/folders/8t/jw61zz954rbcbps32kpn49l80000gn/T/ipykernel_3062/2330918401.py:43: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  image_np = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:301: RuntimeWarning: invalid value encountered in divide
  return band / np.percentile(band, percentile)
/var/folders/8t/jw61zz954rbcbps32kpn49l80000gn/T/ipykernel_3062/2330918401.py:59: RuntimeWarning: invalid value encountered in divide
  image_8bit = ((image - np.min(image)) /
/var/folders/8t/jw61zz954rbcbps32kpn49l80000gn/T/ipykernel_3062/2330918401.py:60: RuntimeWarning: invalid value encountered in cast
  (np.max(image) - np.min(image)) * 255).astype('uint8')
/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:616: RuntimeWarning: invalid value encountered in divide
  NDB

In [49]:
numerical_df = pd.DataFrame(columns=['ID', 'split', 'Label', 'NDVI', 'NBR', 'NDWI', 'NDBI', 'RGB'])

# Instantiate the dataset
for split in ['test', 'train', 'val']:
    df = df_sentinel2[df_sentinel2['split'] == split]
    sat_dataset = dataset.SATDataset(split=split,
                                    data_path=Path("../prepared_dataset"),
                                    df=df)
    new_df = save_all_numerical_values_to_df(split, sat_dataset)
    numerical_df = pd.concat([numerical_df, new_df])
    # add the calculated_df to the numerical_df
    # numerical_df = pd.concat([numerical_df, calculated_df])


Filepath: image_directory_20210712_5
Location info: image_directory_20210712_5
Filepath: image_directory_20210628_6
Location info: image_directory_20210628_6
Filepath: image_directory_20210627_32
Location info: image_directory_20210627_32
Filepath: image_directory_20210413_33


/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:301: RuntimeWarning: invalid value encountered in divide
  return band / np.percentile(band, percentile)
/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:301: RuntimeWarning: invalid value encountered in divide
  return band / np.percentile(band, percentile)
/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:301: RuntimeWarning: invalid value encountered in divide
  return band / np.percentile(band, percentile)


Location info: image_directory_20210413_33
Filepath: image_directory_20210730_34
Location info: image_directory_20210730_34
Filepath: image_directory_20210819_35
Location info: image_directory_20210819_35
Filepath: image_directory_20210418_36
Location info: image_directory_20210418_36
Filepath: image_directory_20210818_37
Location info: image_directory_20210818_37
Filepath: image_directory_20210627_38
Location info: image_directory_20210627_38
Filepath: image_directory_20210819_39
Location info: image_directory_20210819_39
Filepath: image_directory_20210627_0
Location info: image_directory_20210627_0
Filepath: image_directory_20210628_1
Location info: image_directory_20210628_1
Filepath: image_directory_20210628_2
Location info: image_directory_20210628_2
Filepath: image_directory_20210513_3


/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:672: RuntimeWarning: invalid value encountered in divide
  NDVI = (B08 - B04) / (B08 + B04)
/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:674: RuntimeWarning: invalid value encountered in divide
  NBR = (B08 - B07_upsampled) / (B08 + B07_upsampled)
/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:683: RuntimeWarning: invalid value encountered in divide
  NDBI = (B11 - B08) / (B11 + B08)
/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:301: RuntimeWarning: invalid value encountered in divide
  return band / np.percentile(band, percentile)


Location info: image_directory_20210513_3
Filepath: image_directory_20210418_4
Location info: image_directory_20210418_4
Filepath: image_directory_20210818_9
Location info: image_directory_20210818_9
Filepath: image_directory_20210811_10
Location info: image_directory_20210811_10
Filepath: image_directory_20210418_11
Location info: image_directory_20210418_11
Filepath: image_directory_20210515_12
Location info: image_directory_20210515_12
Filepath: image_directory_20210719_13
Location info: image_directory_20210719_13
Filepath: image_directory_20210712_14
Location info: image_directory_20210712_14
Filepath: image_directory_20210413_15
Location info: image_directory_20210413_15
Filepath: image_directory_20210714_16
Location info: image_directory_20210714_16
Filepath: image_directory_20210714_17


...

Location info: image_directory_20210714_17
Filepath: image_directory_20210515_18
Location info: image_directory_20210515_18
Filepath: image_directory_20210627_19
Location info: image_directory_20210627_19
Filepath: image_directory_20210418_20
Location info: image_directory_20210418_20
Filepath: image_directory_20210521_21
Location info: image_directory_20210521_21
Filepath: image_directory_20210513_22


/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:301: RuntimeWarning: invalid value encountered in divide
  return band / np.percentile(band, percentile)


Location info: image_directory_20210513_22
Filepath: image_directory_20210627_23
Location info: image_directory_20210627_23
Filepath: image_directory_20210413_24
Location info: image_directory_20210413_24
Filepath: image_directory_20210811_25
Location info: image_directory_20210811_25
Filepath: image_directory_20210413_26
Location info: image_directory_20210413_26
Filepath: image_directory_20210712_27
Location info: image_directory_20210712_27
Filepath: image_directory_20210418_28
Location info: image_directory_20210418_28
Filepath: image_directory_20210413_29
Location info: image_directory_20210413_29
Filepath: image_directory_20210418_30
Location info: image_directory_20210418_30
Filepath: image_directory_20210902_31
Location info: image_directory_20210902_31
Filepath: image_directory_20210628_7
Location info: image_directory_20210628_7
Filepath: image_directory_20210628_8
Location info: image_directory_20210628_8
Filepath: image_directory_20210413_40
Location info: image_directory_2

/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:672: RuntimeWarning: invalid value encountered in divide
  NDVI = (B08 - B04) / (B08 + B04)
/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:674: RuntimeWarning: invalid value encountered in divide
  NBR = (B08 - B07_upsampled) / (B08 + B07_upsampled)
/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:683: RuntimeWarning: invalid value encountered in divide
  NDBI = (B11 - B08) / (B11 + B08)
/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:672: RuntimeWarning: invalid value encountered in divide
  NDVI = (B08 - B04) / (B08 + B04)
/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:674: RuntimeWarning: invalid value encountered in divide
  NBR = (B08 - B07_upsampled) / (B08 + B07_upsampled)
/Users/glenn_hyh/Documents/github/bc-wildfire-prediction/notebooks/dataset.py:683: RuntimeWarning: invalid value

Location info: image_directory_20210627_41
Filepath: image_directory_20210627_42
Location info: image_directory_20210627_42
Filepath: image_directory_20210625_43
Location info: image_directory_20210625_43
Filepath: image_directory_20210413_44
Location info: image_directory_20210413_44
Filepath: image_directory_20210515_45
Location info: image_directory_20210515_45
Filepath: image_directory_20210712_46
Location info: image_directory_20210712_46
Filepath: image_directory_20210902_47
Location info: image_directory_20210902_47


In [50]:
numerical_df

,ID,split,Label,NDVI,NBR,NDWI,NDBI,RGB
0,20210712_5,test,,"[0.34948628707247276, 0.6085697437187708, 0.52...","[-0.024112584638619018, -0.10084910323924015, ...","[-0.08692643214356115, -0.2542880740655925, -0...","[-0.1362474725838616, -0.038150364653090615, -...","[0.33984635601958046, 0.5922434387140069, 0.46..."
1,20210628_6,test,,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,20210627_32,test,,"[0.0028804697293569417, 0.001420230825822053, ...","[-0.01749824334832954, -0.008235021994741557, ...","[-0.008428411391242932, -0.007911725179138707,...","[-0.9037221070904717, -0.9064051589912484, -0....","[0.9567218850476554, 0.946164525941012, 0.9301..."
3,20210413_33,test,,"[0.8494133519996173, 0.8537370953241261, 0.790...","[0.03902861243854533, 0.06258702451021977, -0....","[-0.7883962289998268, -0.7759254128499745, -0....","[0.22217885802791199, 0.2274783058595916, 0.35...","[0.03488436091535268, 0.05069182783423319, 0.0..."
4,20210730_34,test,,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
5,20210819_35,test,,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
6,20210418_36,test,,"[0.6612346360131921, 0.6496522703236803, 0.637...","[-0.01362548480868294, -0.012863914294700057, ...","[-0.4174133063939522, -0.40080979768088776, -0...","[-0.28810898296020726, -0.28649334413674304, -...","[0.12400777548703278, 0.24994528526681756, 0.1..."
7,20210818_37,test,,"[-0.17022160587889357, 0.09180178117640887, 0....","[-0.03267887110694314, -0.02133600189800813, -...","[0.16310648475328435, -0.015898363584168664, -...","[0.1054572343181659, 0.03307990824353959, -0.0...","[1.0267423557239455, 1.0118235019719837, 0.938..."
8,20210627_38,test,,"[0.875114361439849, 0.8744288378149143, 0.8345...","[-0.005556236528511386, -0.0022066782480317858...","[-0.7481773181042608, -0.7558607273960137, -0....","[-0.017107038671486312, 0.007983799855028055, ...","[0.02846116290035032, 0.06155697445927329, 0.0..."
9,20210819_39,test,,"[0.18926751021205185, 0.29529396298007987, 0.1...","[0.019569408498767694, -0.02396986139518314, 0...","[-0.020502222497971213, -0.05604025048148082, ...","[-0.3089457036720308, -0.15365965213651958, -0...","[0.36969717401113106, 0.5205202237249404, 0.47..."


In [51]:
# add a column called ID to df_sentinel2
# the value is convert from column data_path
# by removing the "image_directory_" prefix
df_sentinel2.insert(0, "ID", df_sentinel2['data_path'].str.replace("image_directory_", ""))

In [52]:
# If the row in df_sentinel2 has same ID as the row in numerical_df
# then give the if)fire column in df_sentinel2 the same value as the Label column in numerical_df

for index, row in df_sentinel2.iterrows():
    label = row['if_fire']
    id = row['ID']
    if numerical_df['ID'].isin([id]).any():
        numerical_df.loc[numerical_df['ID'] == id, 'Label'] = label


In [53]:
numerical_df.sort_values(by='ID', inplace=True)

In [54]:
# print the row where the Label is True
numerical_df[numerical_df['Label'] == True]

,ID,split,Label,NDVI,NBR,NDWI,NDBI,RGB
4,20210418_4,train,True,"[0.6943357305952574, 0.646249976007232, 0.8626...","[0.025419227303949812, 0.01348667784205144, 0....","[-0.6755778590201034, -0.6364052821386085, -0....","[0.4314724413381488, 0.4116416966268579, 0.294...","[0.05988626645360408, 0.06427290150214102, 0.0..."
3,20210513_3,train,True,"[0.5120511498455067, 0.332341761882925, -0.477...","[0.20284124996008096, -0.10929117859688005, -0...","[-0.2718350875558304, -0.0865174615691588, 0.7...","[-0.2974263241393414, -0.00042895714039807836,...","[0.2059861390670427, 0.3654509768942575, 0.266..."
0,20210627_0,train,True,"[0.43046203176957265, 0.48298496407413377, -0....","[0.03619587238164594, -0.02962074717402738, 0....","[-0.13890947892351585, -0.15223037207423518, 0...","[-0.3628150744827507, -0.13542886052148545, -0...","[0.27076672271019697, 0.5141721420487739, 0.37..."
1,20210628_1,train,True,"[0.03783567283277804, 0.1212809418179947, 0.23...","[-0.08305747512045074, -0.04581540190134012, -...","[0.09702862345012588, 0.05040355247181026, 0.0...","[0.002494833441941488, -0.048949069446787306, ...","[0.43733579337088757, 0.5731104692221004, 0.54..."
2,20210628_2,train,True,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,20210628_6,test,True,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
0,20210628_7,val,True,"[0.15094188402362366, 0.15259790299929496, 0.2...","[0.00388429624489767, 0.010458387939522576, 0....","[0.0019407679392088393, -0.015357176541189493,...","[-0.05270984919919856, -0.07844622064416179, -...","[0.42769238076330796, 0.5820137197272428, 0.52..."
1,20210628_8,val,True,"[0.14721412074761392, 0.2016007529095074, 0.35...","[0.051094590629175346, -0.0025431791981781497,...","[-0.014953829570451846, -0.0019725421048271386...","[-0.08677405430140397, -0.08355550794757373, -...","[0.44672541066936744, 0.583250930809382, 0.513..."
0,20210712_5,test,True,"[0.34948628707247276, 0.6085697437187708, 0.52...","[-0.024112584638619018, -0.10084910323924015, ...","[-0.08692643214356115, -0.2542880740655925, -0...","[-0.1362474725838616, -0.038150364653090615, -...","[0.33984635601958046, 0.5922434387140069, 0.46..."


In [55]:
df_sentinel2.sort_values(by='ID', inplace=True)
df_sentinel2[df_sentinel2['if_fire'] == True]

,ID,split,imagery_id,date,coordinates,if_fire,data_path,mask_path
4,20210418_4,train,S2A_9UYR_20210418_1_L2A,2021-04-18,POLYGON ((-126.12112636240417 49.6876281074014...,True,image_directory_20210418_4,mask_directory_20210418_4
3,20210513_3,train,S2B_9UXR_20210513_1_L2A,2021-05-13,POLYGON ((-126.63108549894987 50.1827852845182...,True,image_directory_20210513_3,mask_directory_20210513_3
0,20210627_0,train,S2A_9UYR_20210627_0_L2A,2021-06-27,POLYGON ((-125.16477804389937 49.6598726202365...,True,image_directory_20210627_0,mask_directory_20210627_0
1,20210628_1,train,S2B_9UWR_20210628_1_L2A,2021-06-28,POLYGON ((-127.59119010461468 50.5437699307909...,True,image_directory_20210628_1,mask_directory_20210628_1
2,20210628_2,train,S2B_9UXS_20210628_1_L2A,2021-06-28,POLYGON ((-127.3671432187832 50.68006747113423...,True,image_directory_20210628_2,mask_directory_20210628_2
6,20210628_6,test,S2B_9UXS_20210628_1_L2A,2021-06-28,POLYGON ((-127.26103949458563 50.6094595867684...,True,image_directory_20210628_6,mask_directory_20210628_6
7,20210628_7,val,S2B_9UWS_20210628_1_L2A,2021-06-28,POLYGON ((-127.59115795420932 50.5448487890229...,True,image_directory_20210628_7,mask_directory_20210628_7
8,20210628_8,val,S2B_9UXS_20210628_1_L2A,2021-06-28,POLYGON ((-127.5886182856442 50.54481803086994...,True,image_directory_20210628_8,mask_directory_20210628_8
5,20210712_5,test,S2B_9UYR_20210712_0_L2A,2021-07-12,POLYGON ((-125.35650387832956 49.9424725439986...,True,image_directory_20210712_5,mask_directory_20210712_5


In [56]:
numerical_df['Label'].value_counts()

Label
False    39
True      9
Name: count, dtype: int64

In [57]:
numerical_df.to_csv('../dataset_tables/numerical_df_2.csv', index=False)
df_sentinel2.to_csv('../dataset_tables/df_sentinel2_2.csv', index=False)

In [58]:
# read two csv files
# combine the two csv files
numerical_df_1 = pd.read_csv('../dataset_tables/numerical_df_1.csv')
numerical_df_2 = pd.read_csv('../dataset_tables/numerical_df_2.csv')
final_numerical_df = pd.concat([numerical_df_1, numerical_df_2])

In [60]:
final_numerical_df.to_csv('../dataset_tables/final_dataset.csv', index=False)

In [62]:
# print column names
final_numerical_df.columns

Index(['ID', 'split', 'Label', 'NDVI', 'NBR', 'NDWI', 'NDBI', 'RGB'], dtype='object')

In [59]:
final_numerical_df['Label'].value_counts()

Label
False    206
True      45
Name: count, dtype: int64